<a href="https://colab.research.google.com/github/ajayjai30/Battery-Mangement-System-Lead-Acid-Batteries/blob/main/FINAL_IMPLEMENTATION_BMS_(SOH_PREDICTION).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# STEP 1: SETUP & REPAIR
# ==========================================
!git clone https://github.com/ajayjai30/Battery-Mangement-System-Lead-Acid-Batteries.git
%cd Battery-Mangement-System-Lead-Acid-Batteries

!pip install -r requirements.txt
!pip install gradio

import os
import shutil
import glob

print("\n🔧 RUNNING AUTO-REPAIR...")

# 1. Fix Folder Capitalization (Model -> models)
if os.path.exists("Model") and not os.path.exists("models"):
    print("   Renaming 'Model' to 'models'...")
    os.rename("Model", "models")

# 2. Ensure Scalers are accessible
scaler_files = glob.glob("**/*scaler_*.pkl", recursive=True)
os.makedirs("scalers", exist_ok=True)

for f in scaler_files:
    dst = os.path.join("scalers", os.path.basename(f))
    if os.path.abspath(f) != os.path.abspath(dst):
        shutil.copy(f, dst)
        print(f"   Moved {os.path.basename(f)} to ./scalers/")

# 3. Verify Model Exists
if os.path.exists("models/soh_model_gpu.keras"):
    print("✅ Model Found: soh_model_gpu.keras")
elif os.path.exists("models/soh_model_robust.keras"):
    print("✅ Model Found: soh_model_robust.keras")
else:
    print("❌ WARNING: Model not found. You may need to run the training pipeline.")

print("✅ Setup Complete.")

In [ ]:
# ==========================================
# STEP 2: DATA EXTRACTION
# ==========================================
import zipfile
import glob
import os
import shutil

print("📦 SETTING UP DATASET...")

zip_files = glob.glob("*.zip") + glob.glob("**/*.zip", recursive=True)

if zip_files:
    target_zip = zip_files[0]
    print(f"   Extracting {target_zip}...")
    with zipfile.ZipFile(target_zip, 'r') as zip_ref:
        zip_ref.extractall(".")
    print("✅ Data Extracted.")
else:
    print("⚠️ No Zip file found. Checking for CSV...")

csv_name = "processed_bms_data.csv"
if os.path.exists(csv_name):
    print(f"✅ Dataset Ready: {csv_name}")
else:
    csvs = glob.glob(f"**/{csv_name}", recursive=True)
    if csvs:
        shutil.copy(csvs[0], csv_name)
        print(f"✅ Dataset found and moved to root: {csv_name}")
    else:
        print("❌ Dataset missing! Simulation mode will fail.")

In [ ]:
%%writefile app_gradio.py
import gradio as gr
import pandas as pd
import time
import threading
import requests
from bms_predictor import BMSPredictor
import os
import glob

# ==============================================================================
# INITIALIZATION
# ==============================================================================
try:
    if not os.path.exists("models"):
        if os.path.exists("../models"):
            os.symlink("../models", "models")
            os.symlink("../scalers", "scalers")

    if not os.path.exists("models"):
        raise FileNotFoundError("Models folder missing")

    predictor = BMSPredictor(model_dir='models', scaler_dir='scalers')
    ai_status = "✅ AI Engine Online"
except Exception as e:
    predictor = None
    ai_status = f"❌ Error: {str(e)}"

# Global State
simulation_running = False
history_soh = []
history_v = []
history_i = []

# ==============================================================================
# DATA GENERATOR (CSV or CLOUD)
# ==============================================================================
def data_loop(mode, csv_path, speed, channel_id, read_key):
    global simulation_running, history_soh

    predictor.reset_history()
    history_soh.clear()

    simulation_running = True

    # --- MODE 1: CSV REPLAY ---
    if mode == "CSV Replay":
        # Auto-find CSV
        if csv_path == "processed_bms_data.csv" and not os.path.exists(csv_path):
            found = glob.glob("**/*processed_bms_data.csv", recursive=True)
            if found: csv_path = found[0]

        if not os.path.exists(csv_path):
            yield "❌ CSV Not Found", "0%", "0V", "0A", None
            return

        df = pd.read_csv(csv_path)

        for i in range(len(df)):
            if not simulation_running: break

            row = df.iloc[i]
            v, c, t = row['Voltage_V'], row['Current_A'], row['Temperature_C']

            # Predict & Yield
            yield process_prediction(v, c, t, f"📂 Replay Row {i}/{len(df)}")
            time.sleep(speed)

    # --- MODE 2: LIVE THINGSPEAK ---
    elif mode == "Live ThingSpeak":
        url = f"https://api.thingspeak.com/channels/{channel_id}/feeds/last.json?api_key={read_key}"
        last_entry_id = None

        while simulation_running:
            try:
                r = requests.get(url, timeout=5).json()
                entry_id = r.get('entry_id')

                if entry_id != last_entry_id:
                    # Assumes Field1=Voltage, Field2=Current, Field3=Temp
                    v = float(r.get('field1', 0))
                    c = float(r.get('field2', 0))
                    t = float(r.get('field3', 0))

                    yield process_prediction(v, c, t, f"📡 Live Data [ID: {entry_id}]")
                    last_entry_id = entry_id
                else:
                    # No new data yet
                    pass
            except Exception as e:
                yield f"⚠️ Connection Error: {str(e)[:20]}", "Err", "Err", "Err", None

            time.sleep(15) # ThingSpeak limit

    yield "⏹️ Stopped", f"{history_soh[-1] if history_soh else 0:.2f}%", "0V", "0A", None

def process_prediction(v, c, t, status_msg):
    soh = predictor.predict_realtime(v, c, t)

    if soh:
        history_soh.append(soh)
        df_plot = pd.DataFrame({"Step": range(len(history_soh)), "SOH": history_soh})
        return status_msg, f"{soh:.2f}%", f"{v:.2f} V", f"{c:.2f} A", df_plot
    else:
        return f"{status_msg} (Buffering...)", "--", f"{v}V", f"{c}A", None

def stop_simulation():
    global simulation_running
    simulation_running = False
    return "⏹️ Stopping..."

# ==============================================================================
# UI LAYOUT
# ==============================================================================
with gr.Blocks(title="BMS Cloud Monitor", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🔋 AI Battery SOH Monitor (IoT Edition)")
    gr.Markdown(f"**System Status:** {ai_status}")

    with gr.Row():
        # --- LEFT PANEL: CONTROLS ---
        with gr.Column(scale=1):
            gr.Markdown("### 🎛️ Source Settings")

            mode_input = gr.Radio(["CSV Replay", "Live ThingSpeak"], label="Data Source", value="CSV Replay")

            # CSV Options
            with gr.Group(visible=True) as csv_group:
                csv_path = gr.Textbox(value="processed_bms_data.csv", label="Dataset Path")
                speed_input = gr.Slider(0.01, 1.0, value=0.1, label="Replay Speed")

            # ThingSpeak Options
            with gr.Group(visible=False) as ts_group:
                ts_id = gr.Textbox(label="Channel ID", placeholder="e.g. 123456")
                ts_key = gr.Textbox(label="Read API Key", placeholder="e.g. ABC12345")

            # Buttons
            with gr.Row():
                start_btn = gr.Button("▶️ Start", variant="primary")
                stop_btn = gr.Button("⏹️ Stop", variant="stop")

            status_out = gr.Textbox(label="Status", interactive=False)

        # --- RIGHT PANEL: METRICS ---
        with gr.Column(scale=3):
            gr.Markdown("### 📊 Real-time Health")
            with gr.Row():
                soh_box = gr.Textbox(label="Health (SOH)", value="--", elem_id="soh_val")
                volt_box = gr.Textbox(label="Voltage", value="--")
                curr_box = gr.Textbox(label="Current", value="--")

            plot = gr.LinePlot(
                x="Step", y="SOH", title="SOH Trend", width=600, height=300, y_lim=[0, 105]
            )

    # --- INTERACTIVITY ---
    def toggle_inputs(mode):
        return {
            csv_group: gr.update(visible=(mode == "CSV Replay")),
            ts_group: gr.update(visible=(mode == "Live ThingSpeak"))
        }

    mode_input.change(toggle_inputs, inputs=mode_input, outputs=[csv_group, ts_group])

    start_btn.click(
        fn=data_loop,
        inputs=[mode_input, csv_path, speed_input, ts_id, ts_key],
        outputs=[status_out, soh_box, volt_box, curr_box, plot]
    )

    stop_btn.click(fn=stop_simulation, inputs=None, outputs=status_out)

if __name__ == "__main__":
    demo.queue().launch(share=True, inline=False)

In [ ]:
# ==========================================
# STEP 4: LAUNCH GRADIO
# ==========================================
print("🚀 LAUNCHING GRADIO DASHBOARD...")
print("Look for the 'Running on public URL' link below")

# Run the script
!python app_gradio.py